<a href="https://cocl.us/Data_Science_with_Scalla_top"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/SC0103EN/adds/Data_Science_with_Scalla_notebook_top.png" width = 750, align = "center"></a>
 <br/>
<a><img src="https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width="200" align="center"></a>"

# Module 5: Pipeline and Grid Search

## Predicting Grant Applications: Building a Pipeline

### Lesson Objectives

* After completing this lesson, you should be able to:
  - Understand the role of pipelines in spark.ml
  - Use a pipeline to fit a model and make predictions
  - Evaluate the results
  
### Key Concepts

* Transformer
  - an algorithm which transforms one DataFrame into another
* Estimator
  - an algorithm which can be fit on a DataFrame to produce a Transformer
* Parameter
  - there is a common API shared by Transformers and Estimators
* Pipeline
  - chains multiple Transformers together to specify a machine learning workflow
* Evaluator
  - measures the performance of an estimator or pipeline against some metric(s)
  
### Pipelines in spark.ml

* Inspired by the scikit-learn project
* Components:
  - Transformers
  - Estimators
* Properties of components:
  - Transformer.transform() and Estimator.fit() are stateless
  - Each instance of Transformer/Estimator has a unique ID
* A sequence of PipelineStages to be run in a specific order
  - input DataFrame is transformed as it passes through each stage
  - Transformer stages: `transform()` method is called on the DF
  - Estimator stages: `fit()` method is called to produce a Transformer
    - this Transformer becomes part of the Pipeline Model
    - `transform()` method is called on the DF
* Runtime checking is done using the DF's schema before actually running the Pipeline

### Create the Pipeline

load grant data

In [1]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()
import spark.implicits._
import org.apache.spark.sql.functions._

val data = spark.read.
  format("com.databricks.spark.csv").
  option("delimiter", "\t").
  option("header", "true").
  option("inferSchema", "true").
//   load("/resources/data/grantsPeople.csv")
  load("grantsPeople.csv")

data.show()

Intitializing Scala interpreter ...

Spark Web UI available at http://host.docker.internal:4042
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1670056377421)
SparkSession available as 'spark'


+--------------------+----------+---------------+---------+--------------+---------+--------------------+-------------+----------------+-------------+--------+----------+--------+-----------------------------------+--------------------------+----------------------------+----+----+----+----+------------+------------+--------------------+-------------------+
|Grant_Application_ID| RFCD_Code|RFCD_Percentage| SEO_Code|SEO_Percentage|Person_ID|                Role|Year_of_Birth|Country_of_Birth|Home_Language| Dept_No|Faculty_No|With_PHD|No_of_Years_in_Uni_at_Time_of_Grant|Number_of_Successful_Grant|Number_of_Unsuccessful_Grant|  A2|   A|   B|   C|Grant_Status|Sponsor_Code| Contract_Value_Band|Grant_Category_Code|
+--------------------+----------+---------------+---------+--------------+---------+--------------------+-------------+----------------+-------------+--------+----------+--------+-----------------------------------+--------------------------+----------------------------+----+----+-

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@5a5e893a
import spark.implicits._
import org.apache.spark.sql.functions._
data: org.apache.spark.sql.DataFrame = [Grant_Application_ID: int, RFCD_Code: string ... 22 more fields]


create features

In [2]:
val researchers = data.
  withColumn ("phd", data("With_PHD").equalTo("Yes").cast("Int")).
  withColumn ("CI", data("Role").equalTo("CHIEF_INVESTIGATOR").cast("Int")).
  withColumn("paperscore", data("A2") * 4 + data("A") * 3)

val grants = researchers.groupBy("Grant_Application_ID").agg(
  max("Grant_Status").as("Grant_Status"),
  max("Grant_Category_Code").as("Category_Code"),
  max("Contract_Value_Band").as("Value_Band"),
  sum("phd").as("PHDs"),
  when(max(expr("paperscore * CI")).isNull, 0).
    otherwise(max(expr("paperscore * CI"))).as("paperscore"),
  count("*").as("teamsize"),
  when(sum("Number_of_Successful_Grant").isNull, 0).
    otherwise(sum("Number_of_Successful_Grant")).as("successes"),
  when(sum("Number_of_Unsuccessful_Grant").isNull, 0).
    otherwise(sum("Number_of_Unsuccessful_Grant")).as("failures")
)

grants.show()

+--------------------+------------+-------------+--------------------+----+----------+--------+---------+--------+
|Grant_Application_ID|Grant_Status|Category_Code|          Value_Band|PHDs|paperscore|teamsize|successes|failures|
+--------------------+------------+-------------+--------------------+----+----------+--------+---------+--------+
|                   1|           1|  GrantCatUnk|  ContractValueBandA|null|        22|       1|        0|       0|
|                   2|           1|  GrantCat10A|  ContractValueBandB|   1|        60|       1|        0|       0|
|                   3|           1|  GrantCat10B|  ContractValueBandA|   2|        13|       7|        0|       0|
|                   4|           1|  GrantCat10B|  ContractValueBandC|   2|         0|       6|        0|       0|
|                   5|           0|  GrantCat10A|  ContractValueBandA|   1|        12|       1|        0|       0|
|                   6|           1|  GrantCat10A|ContractValueBandUnk|   1|     

researchers: org.apache.spark.sql.DataFrame = [Grant_Application_ID: int, RFCD_Code: string ... 25 more fields]
grants: org.apache.spark.sql.DataFrame = [Grant_Application_ID: int, Grant_Status: int ... 7 more fields]


String Indexer

In [3]:
import org.apache.spark.ml.feature.StringIndexer

val value_band_indexer = new StringIndexer().
  setInputCol("Value_Band").
  setOutputCol("Value_index").
  fit(grants)
  
val category_indexer = new StringIndexer().
  setInputCol("Category_Code").
  setOutputCol("Category_index").
  fit(grants)
  
val label_indexer = new StringIndexer().
  setInputCol("Grant_Status").
  setOutputCol("status").
  fit(grants)


import org.apache.spark.ml.feature.VectorAssembler

val assembler = new VectorAssembler().
  setInputCols(Array(
    "Value_index"
    ,"Category_index"
    ,"PHDs"
    ,"paperscore"
    ,"teamsize"
    ,"successes"
    ,"failures"
  )).setOutputCol("assembled")

import org.apache.spark.ml.feature.StringIndexer
value_band_indexer: org.apache.spark.ml.feature.StringIndexerModel = StringIndexerModel: uid=strIdx_323e09f25168, handleInvalid=error
category_indexer: org.apache.spark.ml.feature.StringIndexerModel = StringIndexerModel: uid=strIdx_7e6d378a83ee, handleInvalid=error
label_indexer: org.apache.spark.ml.feature.StringIndexerModel = StringIndexerModel: uid=strIdx_c0a0b094dbb6, handleInvalid=error
import org.apache.spark.ml.feature.VectorAssembler
assembler: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_0520db91911f, handleInvalid=error, numInputCols=7


  Random Forest Classifier and Pipeline

In [4]:
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.classification.RandomForestClassificationModel

val rf = new RandomForestClassifier().
  setFeaturesCol("assembled").
  setLabelCol("status").
  setSeed(42)

import org.apache.spark.ml.Pipeline
val pipeline = new Pipeline().setStages(Array(
    value_band_indexer,
    category_indexer,
    label_indexer,
    assembler,
    rf)
  )

import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.classification.RandomForestClassificationModel
rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_dad345a36bdd
import org.apache.spark.ml.Pipeline
pipeline: org.apache.spark.ml.Pipeline = pipeline_2435bacc7c29


### Create an Evaluator

In [5]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
val auc_eval = new BinaryClassificationEvaluator().
  setLabelCol("status").
  setRawPredictionCol("rawPrediction")
auc_eval.getMetricName

import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
auc_eval: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = BinaryClassificationEvaluator: uid=binEval_8ed43d89fc6c, metricName=areaUnderROC, numBins=1000
res2: String = areaUnderROC


### Split into Training and Test

In [6]:
val tr = grants.filter("Grant_Application_ID < 6635")
val te = grants.filter("Grant_Application_ID >= 6635")
val training = tr.na.fill(0, Seq("PHDs"))
val test = te.na.fill(0, Seq("PHDs"))

tr: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Grant_Application_ID: int, Grant_Status: int ... 7 more fields]
te: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Grant_Application_ID: int, Grant_Status: int ... 7 more fields]
training: org.apache.spark.sql.DataFrame = [Grant_Application_ID: int, Grant_Status: int ... 7 more fields]
test: org.apache.spark.sql.DataFrame = [Grant_Application_ID: int, Grant_Status: int ... 7 more fields]


### Run and Evaluate the Pipeline

In [7]:
val model = pipeline.fit(training)
val pipeline_results = model.transform(test)
auc_eval.evaluate(pipeline_results)

model: org.apache.spark.ml.PipelineModel = pipeline_2435bacc7c29
pipeline_results: org.apache.spark.sql.DataFrame = [Grant_Application_ID: int, Grant_Status: int ... 14 more fields]
res3: Double = 0.9129422933491597


### About the Authors

[Petro Verkhogliad](https://www.linkedin.com/in/vpetro) is Consulting Manager at Lightbend. He holds a Masters degree in Computer Science with specialization in Intelligent Systems. He is passionate about functional programming and applications of AI.